In [20]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import time

plt.rcParams.update({'font.size': 18})

In [21]:
h5_file = "test_file.h5"

In [22]:
print ("Reading",h5_file)
hf = h5py.File(h5_file, 'r')
print (hf["data/temperature"][()])
hf.close()

Reading test_file.h5
[0.         0.00876522 0.01539326 0.0220933  0.02880979 0.03546429
 0.04211307 0.04842353 0.05446124 0.06009984 0.06517863]
